<img src="https://github.com/jupytercon/2020-exactlyallan/raw/master/images/RAPIDS-header-graphic.png" style="width:50%">


# RAPIDS Visualization Guide Notebook
### A Streamlined Guide to RAPIDS Accelerated Visualization and Visual Analtyics
The guide will walk through using RAPIDS cuDF, cuSpatial, and cuGraph with Holoviews, hvPlot, Datashader, cuxfilter, and Plotly Dash with the publically availble Divvy Bike share dataset. 

**NOTES and TODO:**
-Base on [JupyterCon Notebooks](https://github.com/rapidsai-community/event-notebooks/blob/main/JupyterCon_2020_RAPIDSViz/00%20Index%20and%20Introduction.ipynb) not cuxfilter tutorial


## Requirements
- System that meets the [RAPIDS system and GPU requirements](https://docs.rapids.ai/install#system-req)


## Dependencies
Use the below to install all the required dependencies via conda:



In [ ]:
# imports
import os
from zipfile import ZipFile
from pathlib import Path

import cupy

import cudf
import cuspatial
import cugraph
import cuml

from bokeh.models import NumeralTickFormatter
import hvplot.cudf
hvplot.extension('bokeh')
import colorcet
import panel as pn

import cuxfilter


## Load Dataset
The dataset can be downloaded from the [Divvy Bike Share public dataset](https://divvybikes.com/system-data). Use the following script to download the desired date range and load it into a dataframe.


In [ ]:
# Define the URL of the Divvy trip data and save dir
S3 = 'https://divvy-tripdata.s3.amazonaws.com/'
DATA_DIR = './data'


In [ ]:
# Check dir
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
"""
# Download the zip files from the URL within date range and unzip
for year in range(2021, 2022):
    for month in range(1, 4):
        file = f'{year}{month:02d}-divvy-tripdata.zip'
        URL = f'{S3}{file}'
        ! wget -P {DATA_DIR} {URL}
     
        with ZipFile(f'{DATA_DIR}/{file}') as zip:
            zip.extractall(f'{DATA_DIR}')
"""            

### cuDF

In [ ]:

# Load all csv as dataframes and combine into one cudf
df_array = []

for file in Path(DATA_DIR).rglob('20*.csv'):
    gdf = cudf.read_csv(file)
    df_array.append(gdf)

df = cudf.concat(df_array)

# Check the data
df.reset_index()

## Check and Clean Data

The data seems unreasonabliy clean, but there are still a few things we improve on it. First lets double check the dtypes.


Lets check for blanks and nulls first


In [ ]:
df.isnull().sum()


In [ ]:
# Filter rows with at least one null value
df[df['end_lat'].isnull()]


In [ ]:
# drop nulls
df = df.dropna(subset=['end_lat']).reset_index()
df.isnull().sum()

In [ ]:
# Replace null values in 'start_station_id' with 'none'
df['start_station_name'] = df['start_station_name'].fillna('none')

# Replace null values in 'end_station_id' with 'none'
df['end_station_name'] = df['end_station_name'].fillna('none')


In [ ]:
df.dtypes

The 'started_at' and 'ended_at' columns should be proper date times types.

In [ ]:
df['started_at'] = cudf.to_datetime(df['started_at'])
df['ended_at'] = cudf.to_datetime(df['ended_at'])

df

To make things a bit easier lets break out the date and time into sperate columns, assuming we only need to worry about start time.

In [ ]:
df['year'] = df['started_at'].dt.year
df['month'] = df['started_at'].dt.month
df['day'] = df['started_at'].dt.day
df['hour'] = df['started_at'].dt.hour

df

Extracting out the day of the week would be hepful too.

In [ ]:
df['day_of_week'] = df['started_at'].dt.dayofweek

df

In [ ]:
rider_type = df.groupby('member_casual').size().rename("count").reset_index()
rider_type


### hvPlot

In [ ]:
rider_type.hvplot.bar(x='member_casual', y='count', title='Total Rider Types', yformatter='%0.0f')

In [ ]:
# DOW = {0:'M', 1:'T', 2:'W', 3:'Th', 4:'F', 5:'Sa', 6:'Su'}

day_counts = df.groupby('day_of_week').size().rename('count').reset_index().sort_values('day_of_week')
day_counts.hvplot.bar('day_of_week', 'count', title="Trip starts per Week Day", yformatter="%0.0f")

In [ ]:
# calculated duration in min
df['dur_min'] = (df['ended_at'] - df['started_at'])
df['dur_min'] = (df['dur_min'].dt.seconds / 60).round().astype('float32') #needed for cuML KDE

df

In [ ]:
df.hvplot.hist(y='dur_min', bins=20, title="Trips Duration Histrogram", yformatter="%0.0f")

In [ ]:
trips_by_hour = df.groupby('hour').size().rename('count').reset_index()

avg_duration_by_hour = df.groupby('hour')['dur_min'].mean().rename('duration_mean').reset_index()

trips_by_hour.hvplot.bar('hour', 'count', title="Trip starts, per hour", yformatter="%0.0f") + avg_duration_by_hour.hvplot.bar('hour', 'duration_mean', title="Trip duration, per hour", yformatter="%0.0f") 

In [ ]:
trips_by_hour_300 = df[df['dur_min'] <= 300].groupby('hour').size().rename('count').reset_index()

avg_duration_by_hour_300 = df[df['dur_min'] <= 300].groupby('hour')['dur_min'].mean().rename('duration_mean').reset_index()

trips_by_hour_300.hvplot.bar('hour', 'count', title="Trip starts, per hour", yformatter="%0.0f") + avg_duration_by_hour_300.hvplot.bar('hour', 'duration_mean', title="Trip duration, per hour", yformatter="%0.0f") 

In [ ]:
df.loc[df['dur_min'].argsort().tail(10)]

### cuML + KDE

In [ ]:
# CUML KDE

# start, end, step size
dur_range = cupy.arange(1.0, 200.0, 1.0)

kde = cuml.KernelDensity(kernel='gaussian', bandwidth=1).fit(df['dur_min'])

log_density_values = kde.score_samples(dur_range)
density_values = cupy.exp(log_density_values)

density_df = cudf.DataFrame({'duration': dur_range, 'density': density_values})

density_df.hvplot.line(x='duration', y='density', xlabel='Data', ylabel='Density', title='Duration in min KDE')


In [ ]:
# group data by month, day_of_week and hour, count the number of rows in each group
heatmap_data_dw = df.groupby(['day_of_week','hour']).size().rename('count').reset_index()
heatmap_data_dw.hvplot.heatmap(x='day_of_week', y='hour', C='count') 

In [ ]:
# group data by month, day_of_week and hour, count the number of rows in each group
heatmap_data_dwm = df.groupby(['month','day_of_week','hour']).size().rename('count').reset_index()
heatmap_data_dwm.hvplot.heatmap(x='day_of_week', y='hour', C='count', groupby='month', widget_location='left_top')

In [ ]:
df.hvplot.hexbin(x='start_lng', y='start_lat', cmap=colorcet.bgy, geo=True, tiles="OSM", logz=False, gridsize=150, width=700, height=600) + df.hvplot.hexbin(x='end_lng', y='end_lat', geo=True, cmap=colorcet.bgy, tiles="OSM", logz=False, gridsize=150, width=700, height=600)

And if you look at their system map, the lat longs seem to be accurate https://account.divvybikes.com/map.
But this seems like a lot of start / stop places, lets see if we can identify stations.

In [ ]:
df['start_station_id'].unique()


In [ ]:
df['start_lat'].round(4).unique()


So there are obviously many more starting points than stations, so it must be that the bikes do not have to start and stop at a station. We will have to find a way to bin the start stop locations into a reasonable number.

In [ ]:
bike_type = df.groupby('rideable_type').size().rename('count').reset_index()
bike_type.hvplot.bar(x='rideable_type', y='count', title='Total Bike Types', yformatter='%0.0f')

### hvPlot + Datashader + Panel

In [ ]:
start_elec = df[df['rideable_type'] == 'electric_bike'].hvplot.points(x='start_lng', y='start_lat', geo=True, tiles="CartoDark", width=700, height=500, datashade=True, dynspread=True) 
end_elec = df[df['rideable_type'] == 'electric_bike'].hvplot.points(x='end_lng', y='end_lat', geo=True, tiles="CartoDark", width=700, height=500, datashade=True, dynspread=True) 

start_classic = df[df['rideable_type'] != 'electric_bike'].hvplot.points(x='start_lng', y='start_lat', geo=True, tiles="CartoDark", width=700, height=500, datashade=True, dynspread=True)
end_classic = df[df['rideable_type'] != 'electric_bike'].hvplot.points(x='end_lng', y='end_lat', geo=True, tiles="CartoDark", width=700, height=500, datashade=True, dynspread=True) 

elec_row = pn.Row(start_elec, end_elec)
classic_row = pn.Row(start_classic, end_classic)

fourbyfour = pn.Column(elec_row, classic_row)
fourbyfour

In [ ]:
# get a rough map of stations 
non_elect = df[df['rideable_type'] != 'electric_bike']
non_elect['end_lat'] = non_elect['end_lat'].round(3)
non_elect['end_lng'] = non_elect['end_lng'].round(3)

unique_df = non_elect.drop_duplicates(subset=['end_lng', 'end_lat'])
unique_df

station_count = unique_df.shape[0]
print('Approximate number of bike stations: ', station_count)

raster = df.hvplot.points(x='end_lng', y='end_lat', geo=True, tiles="CartoDark", hover=True, width=700, height=500, rasterize=True) #note: raster does not aggregate with datashader
station_points = unique_df.hvplot.points(x='end_lng', y='end_lat', color='red', alpha=0.3, geo=True, hover=True, datashade=False)
raster * station_points

### cuSpatial

In [ ]:
# Create a cuSpatial GeoSeries from the latitude and longitude columns
start_points = cuspatial.GeoSeries.from_points_xy(df[['start_lng','start_lat']].interleave_columns().astype("float64"))
end_points = cuspatial.GeoSeries.from_points_xy(df[['end_lng','end_lat']].interleave_columns().astype("float64"))


In [ ]:
distances_in_km = cuspatial.haversine_distance(start_points, end_points)
distances_in_km

In [ ]:
# add the distances back into the dataframe, rounding values to make it more obvious if the stopped at the same place it started
dist_m = cudf.Series(distances_in_km).values * 1000
df['dist_m'] = dist_m.round().astype('int16')
df

In [ ]:
df.hvplot.hist(y='dist_m', by='rideable_type', bins=80, title="Trips Distance By Type", yformatter="%0.0f") + df[df['dist_m'] > 0].hvplot.hist(y='dist_m', by='rideable_type', bins=80, title="Trips Distance By Type ( W/O Returns)", yformatter="%0.0f")

### cuxfilter

In [ ]:
# NOTE: Try with NIGHTLY after datatiles dropped
'''
# Specify the charts and widgets to use with the selected columns of data and string maps
cux_df = cuxfilter.DataFrame.from_dataframe(df)

#creating a label map for days of week strings
days_of_week_map = {
    0: 'M',
    1: 'T',
    2: 'W',
    3: 'Th',
    4: 'F',
    5: 'Sa',
    6: 'Su',
    7: 'Unknown'
}

month_of_year = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}


charts = [
    cuxfilter.charts.bar('dist_m', data_points=10 , title='Distance in M'),
    cuxfilter.charts.bar('dur_min', data_points=10 , title='Duration in Min'),
    cuxfilter.charts.bar('day_of_week', x_label_map=days_of_week_map, title='Day of Week'),
    cuxfilter.charts.bar('hour', title='Trips per Hour'),
    cuxfilter.charts.bar('day', title='Trips per Day'),
    #cuxfilter.charts.bar('year', title='Trips per Year') #wait until loading larger datasets / datatiles removed
]

## FIX-NOTE: wait until datatiles removed
"""
widgets = [
    cuxfilter.charts.multi_select('member_casual', label_map=member_map),
    cuxfilter.charts.multi_select('rideable_type', label_map=bike_map)
]
"""

# Generate the dashboard and select a layout
d = cux_df.dashboard(charts, layout=cuxfilter.layouts.two_by_three, theme=cuxfilter.themes.rapids, title='Bike Trips Dashboard')

# Update the yaxis ticker to an easily readable format
for i in charts:
    if hasattr(i.chart, 'yaxis'):
        i.chart.yaxis.formatter = NumeralTickFormatter(format="0,0")

# FIX-NOTE: adding extension here explicitly RELOADS bokeh and all plots will work
hvplot.extension('bokeh')

# show is for seperate dashboard, await d.preview() is to generate an inline image preview, d.app() shows the app inline
d.show()
'''


## cuML + Kmeans

In [ ]:
## TEMP
df.to_parquet('./data/bike_df_clean.parquet') 
df

In [ ]:
# TEMP 
df = cudf.read_parquet('./data/bike_df_clean.parquet')

# combine all lat values 
lat_df = cudf.DataFrame()
lat_df['lat'] = cudf.concat([df['start_lat'], df['end_lat']], ignore_index=True)

# combine all lng values
lng_df = cudf.DataFrame()
lng_df['lng'] = cudf.concat([df['start_lng'], df['end_lng']], ignore_index=True)

# create combined lat lng 
combined_lat_lng_df = cudf.concat([lat_df, lng_df], axis=1)

combined_lat_lng_df


In [ ]:
# Perform k-means clustering, from the approximate station count with a bit of headroom
kmeans = cuml.cluster.KMeans(n_clusters=station_count+20, max_iter=600)
kmeans.fit(combined_lat_lng_df)

# Get the cluster labels
cluster_labels = kmeans.labels_

cluster_labels

In [ ]:
# Get the edge list from t he computed clusters
half_length = len(cluster_labels) // 2

edge_list_df = cudf.DataFrame({
    'src': cluster_labels[:half_length].reset_index(drop=True),
    'dst': cluster_labels[half_length:].reset_index(drop=True)
})

edge_list_df

In [ ]:
# Get the cluster centers or Nodes
node_centers_df = kmeans.cluster_centers_

node_centers_df = node_centers.rename(columns={0: 'node_lat', 1: 'node_lng'}).astype('float32')

node_centers_df

In [ ]:
# Verify the clustering worked by overlaying with the previous datashader chart
cluster_map = node_centers_df.hvplot.points(x='node_lng', y='node_lat', geo=True, width=700, height=500, color='orange', alpha=0.7, hover=True, datashade=False)

end_elec * cluster_map #Note: only specify geo=True, once otherwise the map tiles overlay the data of the second chart

# Note: looks pretty good, within about a 2 block tolerance 

## Clean up and Save

In [ ]:
## TEST 
## TO-DO create some sort of dictionary mapping the node_lat and node_long values from edge list src dst

# Update start_lat and start_lng columns
df['start_lat'] = df['src'].map(node_dict).str[0]
df['start_lng'] = df['src'].map(node_dict).str[1]

# Update end_lat and end_lng columns
df['end_lat'] = df['dst'].map(node_dict).str[0]
df['end_lng'] = df['dst'].map(node_dict).str[1]



In [ ]:
# Starting to get a big messy, lets clean it up
df_dur = df['dur_min'].astype('int16')

df = df.drop(['index','ride_id','started_at','ended_at','start_lat','start_lng','end_lat','end_lng','start_station_id','end_station_id','dur_min'], axis=1)
df = cudf.concat([df,df_dur,df_geo], axis=1)
df

In [ ]:
# check dtypes
df.dtypes

In [ ]:
## AJAY CODE - wait until nightly
'''
df = cudf.read_parquet('./data/bike_df_clean.parquet')

# 89,870 unique edges
edges = df[['start_grid_id', 'end_grid_id']].drop_duplicates().reset_index(drop=True)

# 140,052 unique nodes
nodes = cudf.concat([
    df[['start_grid_id', 'start_lat', 'start_lng']].rename(columns={'start_grid_id':'grid', 'start_lat':'lat', 'start_lng':'lng'}),
    df[['end_grid_id', 'end_lat', 'end_lng']].rename(columns={'end_grid_id':'grid', 'end_lat':'lat', 'end_lng':'lng'})
]).drop_duplicates().reset_index(drop=True)

cux_df_geo = cuxfilter.DataFrame.load_graph((nodes, edges))

charts = [cuxfilter.charts.graph(
        node_id='grid',
        node_x='lat',
        node_y='lng',
        edge_source='start_grid_id', edge_target='end_grid_id',
        node_aggregate_fn='mean',
        title='Graph for trip source_stations (color by count)',
    )]

d = cux_df_geo.dashboard(charts, layout=cuxfilter.layouts.single_feature, theme=cuxfilter.themes.rapids, title='Geospatial Trips')

d.app()
'''

In [ ]:
# Specifying a graph chart type will use Datashader and its required parameters
'''
charts = [
    cuxfilter.charts.graph(
        node_id='start_grid_id',
        edge_source='src', edge_target='dst',
        node_aggregate_fn='count',
        node_pixel_shade_type='linear', node_point_size=35, #node size is fixed
        edge_render_type='curved', #other option: direct
        edge_transparency=0.7, #0.1 - 0.9
        tile_provider='CARTODBPOSITRON', 
        title='Graph for trip source_stations (color by count)'
    ),
    cuxfilter.charts.graph(
        node_id='end_grid_id',
        edge_source='src', edge_target='dst',
        node_aggregate_fn='count',
        node_pixel_shade_type='linear', node_point_size=35, #node size is fixed
        edge_render_type='direct', #other option: direct
        edge_transparency=0.7, #0.1 - 0.9
        tile_provider='CARTODBPOSITRON', 
        title='Graph for trip source_stations (color by count)'
    ),
    cuxfilter.charts.bar('duration_min', data_points=10 , title='Duration in Min'),
    cuxfilter.charts.bar('day_of_week', x_label_map=days_of_week_map, title='Day of Week'),
    cuxfilter.charts.bar('hour', title='Trips per Hour'),
    cuxfilter.charts.view_dataframe(['rideable_type', 'member_casual','ride_id'], drop_duplicates=True)
]

# Generate the dashboard, select a layout and theme
d = cux_df_geo.dashboard(charts, layout=cuxfilter.layouts.double_feature_quad_base, theme=cuxfilter.themes.rapids, title='Geospatial Trips')

# Update the yaxis ticker to an easily readable format
for i in charts:
    if hasattr(i.chart, 'yaxis'):
        i.chart.yaxis.formatter = NumeralTickFormatter(format="0,0")
        
d.show()
'''

### Plotly Dash

## To Do Outline
- cuGraph PageRank leave, PageRank arrive
- Add Plotly Dash chart

Issues:
- juplyter lab needs to be 3.6.4
- FIX: use latest cuxfilter so can filter by string category
- FIX: cuxfilter / hvplot bokeh.js assets bug
- Issue: graph OOMing somehow (bug in cuxfilter percision) + optimzied node / edges with unique

## Conclusion